In [10]:
import pandas as pd

## DOMA
# Define the file path
nodes_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
roads_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Roads_processed.csv"
bridges_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Bridges_processed.csv"
bridges_selected_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Bridges_selected.csv"
survey_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\surveys\Survey.csv"


# # Práca
# # Define the file path
# nodes_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
# roads_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Roads_processed.csv"
# bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Bridges_processed.csv"
# bridges_selected_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Bridges_selected.csv"
# survey_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\surveys\Survey.csv"

# Load the CSV data into a DataFrame
df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_bridges_sel = pd.read_csv(
    bridges_selected_path, encoding='utf-8', delimiter=',')
df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

C:\Users\Doktor\AppData\Local\Temp\ipykernel_14560\1213450377.py:21: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
C:\Users\Doktor\AppData\Local\Temp\ipykernel_14560\1213450377.py:22: DtypeWarning: Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')


In [11]:
df_bridges_all['Dĺžka_premostenia'] = pd.to_numeric(
    df_bridges_all['Dĺžka_premostenia'], errors='coerce')

In [12]:
df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Most_DC', 'Názov_mosta', 'ID_DC',
       'správcovské_číslo', 'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška',
       'Druh_konštrukcie', 'Material', 'Normálna_zaťažiteľnosť',
       'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
       'Spôsob_stanovenia', 'Predmet_premostenia', 'Rok_postavenia',
       'Zaťažovacia_trieda_STN', 'Počet_otvorov', 'Dĺžka_premostenia',
       'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023'],
      

In [13]:
df_bridges_all["Kraj"].value_counts()

Kraj
Prešov             1851
Banská Bystrica    1633
Žilina             1376
Trenčín             967
Košice              801
Nitra               765
Trnava              534
Bratislava          382
Nezistený           193
Name: count, dtype: int64

In [14]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
filtered_bridges = df_bridges_all[
    (df_bridges_all['Kraj'] == "Banská Bystrica") &
    (df_bridges_all['n_2023'] > 4) &
    ((df_bridges_all['Trieda_PK'] == "cesta II. triedy") |
    (df_bridges_all['Trieda_PK'] == "cesta III. triedy")) &
    (df_bridges_all['Dĺžka_premostenia'] >= 10)
]

# Display the filtered DataFrame
filtered_bridges.shape

(49, 51)

In [15]:
import geopandas as gpd
# Replace the file path with your actual file path
file_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'
file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'

# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(file_path)
# Display the first few rows of the GeoDataFrame
# gdf

In [16]:
import folium


def plot_filtered_bridges(df, gdf, regions):
    # Find the average coordinates for the initial map center
    avg_lat = df['Zemepisná_šírka'].mean()
    avg_long = df['Zemepisná_dĺžka'].mean()

    # Create the map with custom tileset
    m = folium.Map(location=[avg_lat, avg_long],
                   zoom_start=9, tiles="Cartodb positron")

    # Colors for the markers
    colors = ["#ff6600", "#0066FF"]

    # Bridge icon
    bridge_icon_url = "https://raw.githubusercontent.com/Medvedku/Assets/main/markers/bridge_marker_Cartodb_orng_blue.svg"

    # Add a marker for each bridge
    for index, row in df.iterrows():
        ic_size = 50
        bridge_icon = folium.CustomIcon(
            icon_image=bridge_icon_url,
            icon_size=(ic_size, ic_size),
            icon_anchor=(ic_size/2, ic_size)
        )

        popup_html = f"<table style='width:400px;'>"
        popup_html += f"<tr><th style='text-align:left;'>Názov mosta:</th><td>{
            row['Názov_mosta']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Trieda PK:</th><td>{
            row['Trieda_PK']}</td></tr>"
        
        popup_html += "<tr><th style='text-align:left;'>Dĺžka premostenia:</th><td>"
        if pd.notna(row['Dĺžka_premostenia']):
            popup_html += f"{float(row['Dĺžka_premostenia'])} m"
        else:
            popup_html += "Unknown"
        popup_html += "</td></tr>"

        popup_html += f"<tr><th style='text-align:left;'>Druh konštrukcie:</th><td>{
            row['Druh_konštrukcie']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Materiál:</th><td>{
            row['Material']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Rok postavenia:</th><td>{
            row['Rok_postavenia']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>GPS Coordinates:</th><td>({
            row['Zemepisná_šírka']:.4f}, {row['Zemepisná_dĺžka']:.4f})</td></tr>"
        popup_html += f"</table>"

        popup = folium.Popup(popup_html, parse_html=False)

        tooltip_html = f"<table style='width:50px;'>"
        tooltip_html += f"<tr><th colspan='2' style='text-align:left;'>Most {
            row['ID_mosta']}</th></tr>"
        tooltip_html += f"<tr><td style='text-align:left;'>STS:</td><td>{
            row['n_2023']}</td></tr>"
        tooltip_html += f"</table>"

        # Add the marker with the custom icon and popup
        folium.Marker(
            [row['Zemepisná_šírka'], row['Zemepisná_dĺžka']],
            icon=bridge_icon,
            popup=popup,
            tooltip=tooltip_html
        ).add_to(m)

    # Filter the GeoDataFrame to include only the specified regions
    region_gdf = gdf[gdf['NM4'].isin(regions)]

    # Define custom style for the regions' polygons
    def style_function(x): return {
        'fillColor': '#98d5f0', 'color': '#98d5f0', 'weight': 2}

    # Add the boundaries of the specified regions to the map with the custom style
    folium.GeoJson(region_gdf, name='Regions Boundaries',
                   style_function=style_function).add_to(m)

    return m


# Usage example (replace 'filtered_bridges' with your filtered DataFrame and adjust regions as needed)
regions_to_plot = ['Banskobystrický']
map = plot_filtered_bridges(filtered_bridges, gdf, regions_to_plot)
map.save('filtered_bridges.html')
map

SyntaxError: unterminated string literal (detected at line 29) (3337898729.py, line 29)

In [ ]:
filtered_bridges["n_2023"].value_counts()

n_2023
5    26
6    21
7     2
Name: count, dtype: int64

In [ ]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
filtered_bridges = df_bridges_all[
   
    (df_bridges_all['n_2023'] > 5) 

]

# Display the filtered DataFrame
filtered_bridges.shape

(854, 51)

In [ ]:
# Define the bridge name to find
bridge_to_find = "M1892"

# Find the bridge in the DataFrame
bridge_info = df_bridges_all[df_bridges_all['ID_mosta'] == bridge_to_find]

# Check if the bridge was found
if not bridge_info.empty:
    # Print the requested information
    print(f"Správcovské Číslo:\t{bridge_info['správcovské_číslo'].values[0]}")
    print(f"Názov mosta:\t\t{bridge_info['Názov_mosta'].values[0]}")
    print(f"Okres:\t\t\t{bridge_info['Okres'].values[0]}")
    print(f"Kraj:\t\t\t{bridge_info['Kraj'].values[0]}")
    print(f"Rok postavenia:\t\t{bridge_info['Rok_postavenia'].values[0]}")
    print(f"Typ konštrukcie:\t{bridge_info['Druh_konštrukcie'].values[0]}")
    print(f"Materiál:\t\t{bridge_info['Material'].values[0]}")
    print(f"Voľná šírka:\t\t{bridge_info['Volná_šírka_mosta'].values[0]}m")
    print(f"Dĺžka premostenia:\t{bridge_info['Dĺžka_premostenia'].values[0]}m")
    print(f"Trieda PK:\t\t{bridge_info['Trieda_PK'].values[0]}")
    print(f"Správca úseku:\t\t{bridge_info['Správca_úseku'].values[0]}")
    print(f"Číslo PK:\t\t{bridge_info['Číslo_PK'].values[0]}")

    # Add GPS coordinates (latitude and longitude) rounded to 4 decimal places
    lat = round(bridge_info['Zemepisná_šírka'].values[0], 5)
    lon = round(bridge_info['Zemepisná_dĺžka'].values[0], 5)
    print(f"GPS Coordinates:\t({lat}, {lon})")

    # Generate and print the Google Maps link
    google_maps_link = f"https://www.google.com/maps?q={lat},{lon}"
    print(f"Google Maps Link:\n{google_maps_link}")

    # Add condition values from n_2012 up to n_2023
    conditions = [bridge_info[f'n_{year}'].values[0]
                  for year in range(2012, 2024)]
    print(f"Conditions:\t\t{conditions}")
else:
    print(f"Bridge with ID '{bridge_to_find}' not found in the DataFrame.")

NameError: name 'df_bridges_all' is not defined

In [ ]:
bridge_info.iloc[0]["Názov_mosta"]

'2668-3_Most cez Ipeľ v obci Nitra nad Ipľom (571002-003)'

In [ ]:
bridge_info.iloc[0]

ID_mosta                                                                                  M4291
Trieda_PK                                                                     cesta III. triedy
Číslo_PK                                                                                   2668
Most_DC                                                                         Dilatačný celok
Názov_mosta                                   2668-3_Most cez Ipeľ v obci Nitra nad Ipľom (5...
ID_DC                                                                                  M4291.01
správcovské_číslo                                                                             3
Zemepisná_dĺžka                                                                        19.77301
Zemepisná_šírka                                                                        48.31872
Výška                                                                                    182.31
Druh_konštrukcie                        